### PNG encryption with RSA 

In [1]:
import os
import zlib
import struct
import random
import datetime
from classes.rsa import RSA 
from classes.png import PNG 
from Crypto.PublicKey import RSA as CryptoRSA
from Crypto.Cipher import PKCS1_OAEP


### Helper functions

In [2]:
def writeEncrypted(data, name, timestamp):
    filename_no_ext = name[:-4]
    new_name = f"{timestamp}_{filename_no_ext}_encrypted.png"
    with open(new_name, 'wb') as new_file:
        for chunk in data:
            new_file.write(chunk)
    return new_name

def writeDecrypted(data, name):
    filename_no_ext = name[:-14]
    new_name = f"{filename_no_ext}_decrypted.png"
    with open(new_name, 'wb') as new_file:
        for chunk in data:
            new_file.write(chunk)

def saveKeys(keys, name, timestamp):
    filename_no_ext = name[:-4]
    new_name = f"{timestamp}_{filename_no_ext}_keys.txt"
    pubkey = keys[0]
    privkey = keys[1]
    with open(new_name, 'w') as new_file:
        new_file.write(f"Public key:\n{pubkey[0]}\n{pubkey[1]}\n")
        new_file.write(f"Private key:\n{privkey[0]}\n{privkey[1]}\n")
    return new_name

def loadKeys(filename):
    with open(filename, 'r') as keyfile:
        lines = keyfile.readlines()
        n_pub = int(lines[1])
        e_pub = int(lines[2])
        d_priv = int(lines[4])
        e_priv = int(lines[5])

        pubkey = (n_pub, e_pub)
        privkey = (d_priv, e_priv)

        return (pubkey, privkey)  

## Encryption & Decryption demos

#### Keypair generation

In [3]:
keypair = RSA.generateKeypair()
print(keypair)

((515880984304879278894855428807, 65537), (515880984304879278894855428807, 132471139735825397355274766993))


#### Path to image

In [4]:
filename = "linux.png"
img_path = f"media/{filename}"

### Szyfrowanie danych zdekompresowanych

##### Encryption

In [5]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

encrypted_idat_chunks = []
# Calculate max chunk size for your RSA key (e.g., 2048 bits, PKCS#1 v1.5 padding)
key_bytes = (pubkey[0].bit_length() + 7) // 8
max_chunk_size = key_bytes - 11  # for PKCS#1 v1.5 padding

for idat in idat_chunks:
    idat_data = idat[8:-4]
    try:
        decompressed_data = zlib.decompress(idat_data)
    except zlib.error:
        decompressed_data = idat_data

    # Split data into chunks
    chunks = [decompressed_data[i:i+max_chunk_size] for i in range(0, len(decompressed_data), max_chunk_size)]
    encrypted_chunks = []
    for chunk in chunks:
        encrypted_chunk = RSA.encryptData(chunk, pubkey)
        length_bytes = len(chunk).to_bytes(4, 'big')
        encrypted_chunks.append(length_bytes + encrypted_chunk)
    encrypted_uncompressed = b''.join(encrypted_chunks)
    encrypted_compressed = zlib.compress(encrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(encrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + encrypted_compressed))
    new_chunk = new_len + idat_type + encrypted_compressed + new_crc

    encrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in encrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

encrypt_img_path = writeEncrypted(all_chunks, png_name, "decompressed")
keyfile = saveKeys(keypair, png_name, "decompressed")

##### Decryption

In [6]:
png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

keys = loadKeys(keyfile)
privkey = keys[1]

decrypted_idat_chunks = []
for chunk in encrypted_idat_chunks:
    encrypted_data = zlib.decompress(chunk[8:-4])
    decrypted_parts = []
    offset = 0
    while offset < len(encrypted_data):
        chunk_len = int.from_bytes(encrypted_data[offset:offset+4], 'big')
        offset += 4
        encrypted_chunk = encrypted_data[offset:offset+key_bytes]
        offset += key_bytes
        decrypted_chunk = RSA.decryptData(encrypted_chunk, privkey)
        decrypted_parts.append(decrypted_chunk[:chunk_len])
    decrypted_uncompressed = b''.join(decrypted_parts)
    decrypted_compressed = zlib.compress(decrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(decrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + decrypted_compressed))
    new_chunk = new_len + idat_type + decrypted_compressed + new_crc

    decrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in decrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

writeDecrypted(all_chunks, png_name)

##### Data lengths check

In [7]:
for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"orig: {len(i)} encrypted: {len(j)} decrypted: {len(k)}")

orig: 54608 encrypted: 54608 decrypted: 12815


### Szyfrowanie danych bez dekompresji danych z pliku

Czy obie metody są równoważne? - Nie!

##### Encryption

In [8]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

orig_data = []
encrypted_idat_chunks = []
for idat in idat_chunks:
    idat_data = idat[8:-4]

    orig_data.append(idat_data)
    encrypted_data = RSA.encryptData(idat_data, pubkey)

    idat_type = b'IDAT'
    new_len = len(encrypted_data).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + encrypted_data))
    new_chunk = new_len + idat_type + encrypted_data + new_crc

    encrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in encrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
    
all_chunks = crit_chunks_no_data

encrypt_img_path = writeEncrypted(all_chunks, png_name, "compressed")
keyfile = saveKeys(keypair, png_name, "compressed")

##### Decryption

In [9]:
keys = loadKeys(keyfile)
privkey = keys[1]

png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

decrypted_data = []
decrypted_idat_chunks = []
for chunk in encrypted_idat_chunks:
    chunk_data = chunk[8:-4]
    decrypted_chunk_data = RSA.decryptData(chunk_data, privkey)
    decrypted_data.append(decrypted_chunk_data)

    idat_type = b'IDAT'
    new_len = len(decrypted_chunk_data).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + decrypted_chunk_data))
    new_chunk = new_len + idat_type + decrypted_chunk_data + new_crc

    decrypted_idat_chunks.append(new_chunk)


crit_chunks_no_data.insert(0, png.png_signature)
for chunk in decrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
    
all_chunks = crit_chunks_no_data

writeDecrypted(all_chunks, png_name)

##### Data lengths check

In [10]:
for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"orig: {len(i)} encrypted: {len(j)} decrypted: {len(k)}")

orig: 11261 encrypted: 12206 decrypted: 11261


## Porównanie metod szyfrowania: ECB (Electronic CodeBook) oraz CBC (Cypher Clock Chaining)

### ECB

#### Encryption

In [11]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

encrypted_idat_chunks = []
# Calculate max chunk size for your RSA key (e.g., 2048 bits, PKCS#1 v1.5 padding)
key_bytes = (pubkey[0].bit_length() + 7) // 8
max_chunk_size = key_bytes - 11  # for PKCS#1 v1.5 padding

for idat in idat_chunks:
    idat_data = idat[8:-4]
    try:
        decompressed_data = zlib.decompress(idat_data)
    except zlib.error:
        decompressed_data = idat_data

    # Split data into chunks
    chunks = [decompressed_data[i:i+max_chunk_size] for i in range(0, len(decompressed_data), max_chunk_size)]
    encrypted_chunks = []
    for chunk in chunks:
        encrypted_chunk = RSA.encryptECB(chunk, pubkey)
        length_bytes = len(chunk).to_bytes(4, 'big')
        encrypted_chunks.append(length_bytes + encrypted_chunk)
    encrypted_uncompressed = b''.join(encrypted_chunks)
    encrypted_compressed = zlib.compress(encrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(encrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + encrypted_compressed))
    new_chunk = new_len + idat_type + encrypted_compressed + new_crc

    encrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in encrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

encrypt_img_path = writeEncrypted(all_chunks, png_name, "ECB")
keyfile = saveKeys(keypair, png_name, "ECB")

#### Decryption

In [12]:
keys = loadKeys(keyfile)
privkey = keys[1]

png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

decrypted_idat_chunks = []
for chunk in encrypted_idat_chunks:
    encrypted_data = zlib.decompress(chunk[8:-4])
    decrypted_parts = []
    offset = 0
    while offset < len(encrypted_data):
        chunk_len = int.from_bytes(encrypted_data[offset:offset+4], 'big')
        offset += 4
        encrypted_chunk = encrypted_data[offset:offset+key_bytes]
        offset += key_bytes
        decrypted_chunk = RSA.decryptECB(encrypted_chunk, privkey)
        decrypted_parts.append(decrypted_chunk[:chunk_len])
    decrypted_uncompressed = b''.join(decrypted_parts)
    decrypted_compressed = zlib.compress(decrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(decrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + decrypted_compressed))
    new_chunk = new_len + idat_type + decrypted_compressed + new_crc

    decrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in decrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

writeDecrypted(all_chunks, png_name)

In [13]:
for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"orig: {len(i)} encrypted: {len(j)} decrypted: {len(k)}")

orig: 11261 encrypted: 54608 decrypted: 12815


### CBC

#### Encryption

In [14]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

block_size = (pubkey[0].bit_length() + 7) // 8
max_chunk_size = block_size - 11  # for PKCS#1 v1.5 padding

# Generate IV for each file (or per chunk if you want)
iv = random.randbytes(block_size)

encrypted_idat_chunks = []
for idat in idat_chunks:
    idat_data = idat[8:-4]
    try:
        decompressed_data = zlib.decompress(idat_data)
    except zlib.error:
        decompressed_data = idat_data

    # Split data into chunks
    chunks = [decompressed_data[i:i+max_chunk_size] for i in range(0, len(decompressed_data), max_chunk_size)]
    encrypted_chunks = []
    prev = iv
    for chunk in chunks:
        encrypted_chunk = RSA.encryptCBC(chunk, pubkey, prev)
        length_bytes = len(chunk).to_bytes(4, 'big')
        encrypted_chunks.append(length_bytes + encrypted_chunk)
        prev = encrypted_chunk  # CBC chaining

    # Store IV at the start of the encrypted data for decryption
    encrypted_uncompressed = iv + b''.join(encrypted_chunks)
    encrypted_compressed = zlib.compress(encrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(encrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + encrypted_compressed))
    new_chunk = new_len + idat_type + encrypted_compressed + new_crc

    encrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in encrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

encrypt_img_path = writeEncrypted(all_chunks, png_name, "CBC")
keyfile = saveKeys(keypair, png_name, "CBC")


#### Decryption

In [15]:
keys = loadKeys(keyfile)
privkey = keys[1]

png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

decrypted_idat_chunks = []
for chunk in encrypted_idat_chunks:
    encrypted_data = zlib.decompress(chunk[8:-4])
    iv = encrypted_data[:block_size]
    offset = block_size
    decrypted_parts = []
    prev = iv
    while offset < len(encrypted_data):
        chunk_len = int.from_bytes(encrypted_data[offset:offset+4], 'big')
        offset += 4
        encrypted_chunk = encrypted_data[offset:offset+block_size]
        offset += block_size
        decrypted_chunk = RSA.decryptCBC(encrypted_chunk, privkey, prev)
        decrypted_parts.append(decrypted_chunk[:chunk_len])
        prev = encrypted_chunk  # CBC chaining

    decrypted_uncompressed = b''.join(decrypted_parts)
    decrypted_compressed = zlib.compress(decrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(decrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + decrypted_compressed))
    new_chunk = new_len + idat_type + decrypted_compressed + new_crc

    decrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in decrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

writeDecrypted(all_chunks, png_name)

##### Data lengths check

In [16]:
for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"orig: {len(i)} encrypted: {len(j)} decrypted: {len(k)}")

orig: 11261 encrypted: 582758 decrypted: 12815


## Porównanie z gotową implementacją RSA

In [17]:

# ----- Encryption -----

# Generate RSA key using PyCrypto
key = CryptoRSA.generate(2048)
pubkey = key.publickey()

# Process PNG chunks
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

encrypted_idat_chunks = []
for idat in idat_chunks:
    idat_data = idat[8:-4]

    # Try to decompress IDAT data (should work for standard PNGs)
    try:
        decompressed_data = zlib.decompress(idat_data)
    except zlib.error:
        decompressed_data = idat_data  # Use as-is if can't decompress

    # Maximum size for PKCS1_OAEP with 2048-bit key is ~214 bytes
    # Need to chunk the data for encryption
    chunk_size = 214
    chunks = [decompressed_data[i:i+chunk_size] for i in range(0, len(decompressed_data), chunk_size)]
    
    # Initialize the cipher
    cipher = PKCS1_OAEP.new(pubkey)
    
    # Encrypt each chunk and concatenate
    encrypted_chunks = []
    for chunk in chunks:
        encrypted_chunk = cipher.encrypt(chunk)
        # Store length + encrypted data to help with decryption
        length_bytes = len(chunk).to_bytes(4, 'big')
        encrypted_chunks.append(length_bytes + encrypted_chunk)
    
    encrypted_uncompressed = b''.join(encrypted_chunks)
    encrypted_compressed = zlib.compress(encrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(encrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + encrypted_compressed))
    new_chunk = new_len + idat_type + encrypted_compressed + new_crc

    encrypted_idat_chunks.append(new_chunk)

# Reconstruct PNG with encrypted chunks
crit_chunks_no_data.insert(0, png.png_signature)
for chunk in encrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

encrypt_img_path = writeEncrypted(all_chunks, png_name, "Crypto")
keyfile = f"Crypto_{png_name[:-4]}_key.pem"
with open(keyfile, "wb") as f:
    f.write(key.export_key("PEM"))

In [18]:
# ----- Decryption -----

# Load the private key
with open(keyfile, 'rb') as f:
    key = CryptoRSA.import_key(f.read())

# Process encrypted PNG
png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

decrypted_idat_chunks = []
for chunk in encrypted_idat_chunks:
    # 1. Decompress the encrypted data
    encrypted_data = zlib.decompress(chunk[8:-4])
    
    # 2. Initialize the decryption cipher
    decipher = PKCS1_OAEP.new(key)
    
    # 3. Process encrypted chunks
    decrypted_parts = []
    offset = 0
    while offset < len(encrypted_data):
        # Read the original chunk length
        chunk_len = int.from_bytes(encrypted_data[offset:offset+4], 'big')
        offset += 4
        
        # The encrypted data is the next 256 bytes (for 2048-bit key)
        encrypted_chunk = encrypted_data[offset:offset+256]
        offset += 256
        
        # Decrypt the chunk
        decrypted_chunk = decipher.decrypt(encrypted_chunk)
        decrypted_parts.append(decrypted_chunk)
    
    # 4. Combine all decrypted parts
    decrypted_uncompressed = b''.join(decrypted_parts)
    
    # 5. Compress the decrypted data to restore PNG format
    decrypted_compressed = zlib.compress(decrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(decrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + decrypted_compressed))
    new_chunk = new_len + idat_type + decrypted_compressed + new_crc

    decrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in decrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

decrypted_path = writeDecrypted(all_chunks, png_name)

for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"Original: {len(i)} bytes | Encrypted: {len(j)} bytes | Decrypted: {len(k)} bytes")

Original: 11261 bytes | Encrypted: 101273 bytes | Decrypted: 12815 bytes


## Porównanie plików

In [19]:
files = ['dice', 'test', 'plte']

for file in files:
    print(f"File: {file}.png")
    png = PNG(f"media/{file}.png")
    anc_chunks = png.getAncillaryChunks()
    crit_chunks = png.getCriticalChunks()
    print(f"Crit chunks: {len(crit_chunks)} Anc chunks: {len(anc_chunks)}")
    print("Crit chunk sizes: ")
    for chunk in crit_chunks:
        data_len = int.from_bytes(chunk[:4])
        chunk_type = chunk[4:8].decode('utf-8')
        print(f"{chunk_type}: {data_len}")


File: dice.png
Crit chunks: 3 Anc chunks: 0
Crit chunk sizes: 
IHDR: 13
IDAT: 179502
IEND: 0
File: test.png
Crit chunks: 10 Anc chunks: 8
Crit chunk sizes: 
IHDR: 13
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 7564
IEND: 0
File: plte.png
Crit chunks: 5 Anc chunks: 6
Crit chunk sizes: 
IHDR: 13
PLTE: 768
IDAT: 8192
IDAT: 2484
IEND: 0
